In [1]:
import tensorflow as tf

2022-12-09 14:51:15.478045: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 14:51:15.559246: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 14:51:15.559259: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-09 14:51:15.984853: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [7]:
d = [
    [1, 2, 3, 4],
    [-1, -2, -3, -4],
    [2, 4, 6, 8],
    [-2, -4, -6, -8]
]

In [8]:
t = tf.constant(d)

In [9]:
t

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[ 1,  2,  3,  4],
       [-1, -2, -3, -4],
       [ 2,  4,  6,  8],
       [-2, -4, -6, -8]], dtype=int32)>

In [18]:
scale_tensor = tf.math.reduce_max(tf.math.abs(t), axis=0, keepdims=False)

In [19]:
t / scale_tensor

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[ 0.5,  0.5,  0.5,  0.5],
       [-0.5, -0.5, -0.5, -0.5],
       [ 1. ,  1. ,  1. ,  1. ],
       [-1. , -1. , -1. , -1. ]])>

In [20]:
tf.math.divide(t, scale_tensor)

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[ 0.5,  0.5,  0.5,  0.5],
       [-0.5, -0.5, -0.5, -0.5],
       [ 1. ,  1. ,  1. ,  1. ],
       [-1. , -1. , -1. , -1. ]])>

In [24]:
t[t[0][0]]

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([-1, -2, -3, -4], dtype=int32)>

In [25]:
t

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[ 1,  2,  3,  4],
       [-1, -2, -3, -4],
       [ 2,  4,  6,  8],
       [-2, -4, -6, -8]], dtype=int32)>